# Analisis de 16s 

## Prefiltrado de los raw Data

In [10]:
%%bash
cd ../3_Days_5_Days_Bacteria_Raw_Fastq/

#Este script nos sirve para reemplazar los espacios por un un guion en el nombre a todos los archivos que lo tengan. 
for file in *; do mv "$file" `echo $file | tr ' ' '_'`; done


In [11]:
#Script nos sirve para cambiar el nombre de los archivos
count=1
for name in *; do mv $name barcode${count}; count=$((count+1)); done


In [15]:
## Unimos los archivos de cada carpeta utilizando el comando "cat"

#cat file1.fastq.gz file2.fastqz.gz > all-file.fastq.fz

#Por cada carpeta se crearon 4 archivos y contienen todos los archivos fastq de cada carptea

### ¿Cuántas secuencias tiene cada archivo?

In [2]:
%%bash
cd ../3_Days_5_Days_Bacteria_Raw_Fastq/all-barcode/
awk '{s++}END{print s/4}' barcode1.fastq #8116
awk '{s++}END{print s/4}' barcode2.fastq #5397
awk '{s++}END{print s/4}' barcode3.fastq #4722
awk '{s++}END{print s/4}' barcode4.fastq #5282


8116
5397
4722
5282


## PORECHOP

In [ ]:
%%bash

#Utilizamos porechop para hacer el recorte de los barcodes

porechop -i /media/betterlab/Datos01/abel/16s-koray/3_Days_5_Days_Bacteria_Raw_Fastq/all-barcode/ -b porechop/ --extra_end_trim 45


### ¿Cuántas secuencias tiene cada archivo, despues de haber sido trimeadas por porechop?

In [3]:
%%bash 

cd ../3_Days_5_Days_Bacteria_Raw_Fastq/porechop/
ls
for file in *; do  echo $file; awk '{s++}END{print s/4}' $file; done 


BC01.fastq
BC02.fastq
BC03.fastq
BC04.fastq
none.fastq
BC01.fastq
7970
BC02.fastq
5244
BC03.fastq
4565
BC04.fastq
5087
none.fastq
120


## Filtrado de secuencias 1200-1800pb

## verificar si estamos en la carpeta

In [5]:
%%bash
cd ../3_Days_5_Days_Bacteria_Raw_Fastq/
ls
awk 'BEGIN {OFS = "\n"} {header = $0 ; getline seq ; getline qheader ; getline qseq ; if (length(seq) >= 1200 && length(seq) <= 1800) {print header, seq, qheader, qseq}}' < porechop/BC01.fastq > 2-length_filtz/Z1_16S_filt.fastq
awk 'BEGIN {OFS = "\n"} {header = $0 ; getline seq ; getline qheader ; getline qseq ; if (length(seq) >= 1200 && length(seq) <= 1800) {print header, seq, qheader, qseq}}' < porechop/BC02.fastq > 2-length_filtz/Z2_16S_filt.fastq
awk 'BEGIN {OFS = "\n"} {header = $0 ; getline seq ; getline qheader ; getline qseq ; if (length(seq) >= 1200 && length(seq) <= 1800) {print header, seq, qheader, qseq}}' < porechop/BC03.fastq > 2-length_filtz/Z3_16S_filt.fastq
awk 'BEGIN {OFS = "\n"} {header = $0 ; getline seq ; getline qheader ; getline qseq ; if (length(seq) >= 1200 && length(seq) <= 1800) {print header, seq, qheader, qseq}}' < porechop/BC04.fastq > 2-length_filtz/Z4_16S_filt.fastq
awk 'BEGIN {OFS = "\n"} {header = $0 ; getline seq ; getline qheader ; getline qseq ; if (length(seq) >= 1200 && length(seq) <= 1800) {print header, seq, qheader, qseq}}' < porechop/none.fastq > 2-length_filtz/Z5_16S_filt.fastq


2-length_filtz
3-chimera_filt
all-barcode
barcode1
barcode2
barcode3
barcode4
porechop


## Utilizaremos awk para el filtrado de los reads

In [10]:
%%bash
#ls 3_Days_5_Days_Bacteria_Raw_Fastq/2-length_filtz/
cd ..
awk '{s++}END{print s/4}' 3_Days_5_Days_Bacteria_Raw_Fastq/2-length_filtz/Z1_16S_filt.fastq
awk '{s++}END{print s/4}' 3_Days_5_Days_Bacteria_Raw_Fastq/2-length_filtz/Z2_16S_filt.fastq
awk '{s++}END{print s/4}' 3_Days_5_Days_Bacteria_Raw_Fastq/2-length_filtz/Z3_16S_filt.fastq
awk '{s++}END{print s/4}' 3_Days_5_Days_Bacteria_Raw_Fastq/2-length_filtz/Z4_16S_filt.fastq

7037
4405
4048
4212


## MINIMAP

In [15]:
%%bash
cd ..

minimap2 -cx map-ont Bd-16s/mock_db.fasta 3_Days_5_Days_Bacteria_Raw_Fastq/2-length_filtz/Z1_16S_filt.fastq -z 70 > 3_Days_5_Days_Bacteria_Raw_Fastq/3-chimera_filt/Z1_16S_filt_align.paf
minimap2 -cx map-ont Bd-16s/mock_db.fasta 3_Days_5_Days_Bacteria_Raw_Fastq/2-length_filtz/Z2_16S_filt.fastq -z 70 > 3_Days_5_Days_Bacteria_Raw_Fastq/3-chimera_filt/Z2_16S_filt_align.paf
minimap2 -cx map-ont Bd-16s/mock_db.fasta 3_Days_5_Days_Bacteria_Raw_Fastq/2-length_filtz/Z3_16S_filt.fastq -z 70 > 3_Days_5_Days_Bacteria_Raw_Fastq/3-chimera_filt/Z3_16S_filt_align.paf
minimap2 -cx map-ont Bd-16s/mock_db.fasta 3_Days_5_Days_Bacteria_Raw_Fastq/2-length_filtz/Z4_16S_filt.fastq -z 70 > 3_Days_5_Days_Bacteria_Raw_Fastq/3-chimera_filt/Z4_16S_filt_align.paf

[M::mm_idx_gen::2.613*1.29] collected minimizers
[M::mm_idx_gen::3.204*1.60] sorted minimizers
[M::main::3.204*1.60] loaded/built the index for 7238 target sequence(s)
[M::mm_mapopt_update::3.509*1.55] mid_occ = 26
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 7238
[M::mm_idx_stat::3.711*1.52] distinct minimizers: 18477244 (73.11% are singletons); average occurrences: 1.398; average spacing: 5.354
[M::worker_pipeline::12.641*2.54] mapped 7037 sequences
[M::main] Version: 2.15-r905
[M::main] CMD: minimap2 -cx map-ont -z 70 Bd-16s/mock_db.fasta 3_Days_5_Days_Bacteria_Raw_Fastq/2-length_filtz/Z1_16S_filt.fastq
[M::main] Real time: 12.669 sec; CPU: 32.140 sec; Peak RSS: 1.121 GB
[M::mm_idx_gen::2.570*1.29] collected minimizers
[M::mm_idx_gen::3.155*1.61] sorted minimizers
[M::main::3.155*1.61] loaded/built the index for 7238 target sequence(s)
[M::mm_mapopt_update::3.469*1.55] mid_occ = 26
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 7238
[M::mm_idx_stat::3.677*1

## YACRD

In [18]:
%%bash 
cd ..
yacrd chimeric -i 3_Days_5_Days_Bacteria_Raw_Fastq/3-chimera_filt/Z1_16S_filt_align.paf -f 3_Days_5_Days_Bacteria_Raw_Fastq/2-length_filtz/Z1_16S_filt.fastq > 3_Days_5_Days_Bacteria_Raw_Fastq/3-chimera_filt/Z1_16S_filt.yacrd
yacrd chimeric -i 3_Days_5_Days_Bacteria_Raw_Fastq/3-chimera_filt/Z2_16S_filt_align.paf -f 3_Days_5_Days_Bacteria_Raw_Fastq/2-length_filtz/Z2_16S_filt.fastq > 3_Days_5_Days_Bacteria_Raw_Fastq/3-chimera_filt/Z2_16S_filt.yacrd
yacrd chimeric -i 3_Days_5_Days_Bacteria_Raw_Fastq/3-chimera_filt/Z3_16S_filt_align.paf -f 3_Days_5_Days_Bacteria_Raw_Fastq/2-length_filtz/Z3_16S_filt.fastq > 3_Days_5_Days_Bacteria_Raw_Fastq/3-chimera_filt/Z3_16S_filt.yacrd
yacrd chimeric -i 3_Days_5_Days_Bacteria_Raw_Fastq/3-chimera_filt/Z4_16S_filt_align.paf -f 3_Days_5_Days_Bacteria_Raw_Fastq/2-length_filtz/Z4_16S_filt.fastq > 3_Days_5_Days_Bacteria_Raw_Fastq/3-chimera_filt/Z4_16S_filt.yacrd

# Conteo

In [22]:
%%bash
cd ..

cd 3_Days_5_Days_Bacteria_Raw_Fastq/2-length_filtz
awk '{s++}END{print s/4}' Z1_16S_filt_filtered.fastq
awk '{s++}END{print s/4}' Z2_16S_filt_filtered.fastq
awk '{s++}END{print s/4}' Z3_16S_filt_filtered.fastq
awk '{s++}END{print s/4}' Z4_16S_filt_filtered.fastq

7006
4399
4042
4206


# MINIMAP2

In [24]:
%%bash
cd ../3_Days_5_Days_Bacteria_Raw_Fastq
ls
minimap2 -cx map-ont ../Bd-16s/mock_db.fasta 2-length_filtz/Z1_16S_filt_filtered.fastq -z 70 > 3-chimera_filt/Z1_16S_align_mock.paf
minimap2 -cx map-ont ../Bd-16s/mock_db.fasta 2-length_filtz/Z2_16S_filt_filtered.fastq -z 70 > 3-chimera_filt/Z2_16S_align_mock.paf
minimap2 -cx map-ont ../Bd-16s/mock_db.fasta 2-length_filtz/Z3_16S_filt_filtered.fastq -z 70 > 3-chimera_filt/Z3_16S_align_mock.paf
minimap2 -cx map-ont ../Bd-16s/mock_db.fasta 2-length_filtz/Z4_16S_filt_filtered.fastq -z 70 > 3-chimera_filt/Z4_16S_align_mock.paf

2-length_filtz
3-chimera_filt
all-barcode
barcode1
barcode2
barcode3
barcode4
porechop


[M::mm_idx_gen::2.620*1.28] collected minimizers
[M::mm_idx_gen::3.214*1.60] sorted minimizers
[M::main::3.214*1.60] loaded/built the index for 7238 target sequence(s)
[M::mm_mapopt_update::3.522*1.55] mid_occ = 26
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 7238
[M::mm_idx_stat::3.723*1.52] distinct minimizers: 18477244 (73.11% are singletons); average occurrences: 1.398; average spacing: 5.354
[M::worker_pipeline::12.604*2.54] mapped 7006 sequences
[M::main] Version: 2.15-r905
[M::main] CMD: minimap2 -cx map-ont -z 70 ../Bd-16s/mock_db.fasta 2-length_filtz/Z1_16S_filt_filtered.fastq
[M::main] Real time: 12.624 sec; CPU: 31.972 sec; Peak RSS: 1.118 GB
[M::mm_idx_gen::2.570*1.29] collected minimizers
[M::mm_idx_gen::3.164*1.60] sorted minimizers
[M::main::3.164*1.60] loaded/built the index for 7238 target sequence(s)
[M::mm_mapopt_update::3.470*1.55] mid_occ = 26
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 7238
[M::mm_idx_stat::3.672*1.52] distinct minimiz

# R

In [25]:
library(readr)
Z1_16S <- read_delim("./Z1_16S_BC04_filt_rrn_align.paf",
"\t", escape_double = FALSE, col_names = FALSE,
trim_ws = TRUE)

NameError: name 'library' is not defined